In [ ]:
from pynq import Overlay
import numpy as np
ol = Overlay("YOUR_bit_file.bit") 

In [ ]:
ol?

you can check frequency

In [ ]:
import pynq
print(pynq.ps.Clocks.fclk0_mhz)

In [ ]:
# The name "axi_bram_ctrl_0 1 2 3" might vary depending on how you construct the block diagram in our specific scenario.

bram_sp_arr = ol.axi_bram_ctrl_0.mmio.array
bram_a_arr = ol.axi_bram_ctrl_1.mmio.array
bram_w_arr = ol.axi_bram_ctrl_2.mmio.array
bram_o_arr = ol.axi_bram_ctrl_3.mmio.array

# TODO 
#### You need to create functions for matrix multiplication (matmul_OS, matmul_WS).
#### The parts marked as #TODO are the sections where you need to write the code. It's okay if the code spans multiple lines. The data going into the FPGA (serial_mat1, serial_mat2) should only consist of MxK, NxK (OS) or MxK NxM (WS) sized data for the operation, and it should not be padded before sending it in.

In [ ]:
def matmul_OS (mat1: np.array, mat2: np.array):
    M = mat1.shape[0]
    K = mat1.shape[1]
    N = mat2.shape[1]
    
    assert(mat1.shape[1] == mat2.shape[0])
    assert(M <= 8 and N <= 8 and K <= 8)
    # START
    # modify mat1 and mat2 (not allowed: padding, allowed: reshape, flatten)
    serial_mat1 = mat1.flatten()
    serial_mat2 = mat2.flatten()
    # load data to BRAM (send MxK elements, NxK elements)
    assert(serial_mat1.shape[0] == M*K and serial_mat2.shape[0] == N*K)
    bram_a_arr[0:serial_mat1.shape[0]] = serial_mat1
    bram_w_arr[0:serial_mat2.shape[0]] = serial_mat2
    # set Special Memory to specify M,K,N and mode (1: OS)
    #TODO
    bram_sp_arr[1] = 1
    bram_sp_arr[2] = M
    bram_sp_arr[3] = K
    bram_sp_arr[4] = N
    # start (set sp(addr0) = 1)
    # TODO 
    bram_sp_arr[0] = 1
    #while until sp(addr100) =>1 
    while(bram_sp_arr[25] != 1):
        print('wait')
        
    # return result  
    # TODO  
    result_mat = bram_o_arr[0:M*N]
    result_mat.reshape(M, N)
    return result_mat
    # END


def matmul_WS (mat1: np.array, mat2: np.array):
    M = mat1.shape[0]
    K = mat1.shape[1]
    N = mat2.shape[1]
    assert(mat1.shape[1] == mat2.shape[0])
    assert(K <= 8 and M <= 8 and N <= 8)
    
    # START
    # modify mat1 and amt2
    serial_mat1 = mat1.flatten()
    serial_mat2 = mat2.flatten()
    # load data to BRAM 
    assert(serial_mat1.shape[0] == M*K and serial_mat2.shape[0] == N*M)
    bram_a_arr[0:serial_mat1.shape[0]] = serial_mat1
    bram_w_arr[0:serial_mat2.shape[0]] = serial_mat2
    # set Special Memory to specify M,K,N and mode (1: OS)
    # TODO
    bram_sp_arr[1] = 0
    bram_sp_arr[2] = M
    bram_sp_arr[3] = K
    bram_sp_arr[4] = N
    # start (set sp(addr0) = 1)
    # TODO
    bram_sp_arr[0] = 1
    #while until sp(addr100) =>1 
    while(bram_sp_arr[25] != 1):
        print('wait')
        
    #return result
    # TODO 
    result_mat = bram_o_arr[0:M*N]
    result_mat.reshape(M, N)
    return result_mat
    # END

## TEST code

In [3]:
import random
def test_OS():
    score = 0
    num_test = 0
    ## OUTPUT STATIONARY TEST
    for _ in range (1000):
        M = random.randint(1,8)
        K = random.randint(1,8)
        N = random.randint(1,8)
        num_test = num_test +1
        mat1 = np.random.randint(10,size=(M,K)) 
        mat2 = np.random.randint(10,size=(K,N))    
        if(np.equal(matmul_OS(mat1, mat2),mat1@mat2).all()):
            score = score +1
            
    return score/num_test * 100

In [1]:
def test_WS():
    score = 0
    num_test = 0
    ## WEIGHT STATIONARY TEST
    for _ in range (1000):
        M = random.randint(1,8)
        K = random.randint(1,8)
        N = random.randint(1,8)
        num_test = num_test +1
        mat1 = np.random.randint(10,size=(K,M)) 
        mat2 = np.random.randint(10,size=(M,N)) 
        if(np.equal(matmul_WS(mat1, mat2), mat1@mat2).all()):
            score = score + 1
    return score/num_test*100

In [ ]:
def project2_test():
    print(" SCORE : " + str((test_WS() + test_OS()) * 0.5) + " / 100 " )

Check your score

In [ ]:
project2_test()